## 傾向スコア
- 岩波DS vol.3 CM接触のアプリ利用への因果効果推定（https://github.com/iwanami-datascience/vol3/tree/master/kato%26hoshino）

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

### データの読み込み

In [2]:
data = pd.read_csv('./q_data_x.csv')

In [4]:
data.columns

Index(['cm_dummy', 'gamedummy', 'area_kanto', 'area_keihan', 'area_tokai',
       'area_keihanshin', 'age', 'sex', 'marry_dummy', 'job_dummy1',
       'job_dummy2', 'job_dummy3', 'job_dummy4', 'job_dummy5', 'job_dummy6',
       'job_dummy7', 'job_dummy8', 'inc', 'pmoney', 'fam_str_dummy1',
       'fam_str_dummy2', 'fam_str_dummy3', 'fam_str_dummy4', 'fam_str_dummy5',
       'child_dummy', 'T', 'F1', 'F2', 'F3', 'M1', 'M2', 'M3', 'TVwatch_day',
       'gamesecond', 'gamecount'],
      dtype='object')

In [23]:
X = data[['TVwatch_day', 'age', 'sex', 'marry_dummy', 'child_dummy', 'inc', 'pmoney','area_kanto', 'area_tokai', 'area_keihanshin', 
          'job_dummy1', 'job_dummy2', 'job_dummy3', 'job_dummy4', 'job_dummy5', 'job_dummy6', 'job_dummy7',
          'fam_str_dummy1', 'fam_str_dummy2', 'fam_str_dummy3', 'fam_str_dummy4']]
Z = data['cm_dummy']

### 傾向スコアの推定
- $e(x) = p(Z=1|X)$

In [24]:
lr = LogisticRegression()
lr.fit(X, Z)
ps = lr.predict_proba(X)[:, 1]

C:\Users\The-A\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [25]:
print('AUC = {:.2f}'.format(roc_auc_score(y_true=Z, y_score=ps)))

AUC = 0.79


In [18]:
data = pd.concat([data, pd.DataFrame(ps, columns=['propensity_score'])], axis = 1)

### 因果効果推定

In [19]:
data_treated = data[data['cm_dummy']==1]
data_untreated = data[data['cm_dummy']==0]

- 単純な差  $E(Y_1|Z=1) - E(Y_0|Z=0)$

In [37]:
avgT = np.mean(data_treated['gamedummy'])
avgU = np.mean(data_untreated['gamedummy'])
diff = avgT - avgU
print('diff = {:.4f}'.format(diff))

diff = 0.0022


- 平均処置効果  $ATE = E(Y_1) - E(Y_0)$

In [38]:
E1 = sum((data['cm_dummy']/data['propensity_score'])*data['gamedummy']) / sum(data['cm_dummy']/data['propensity_score'])
E2 = sum(((1-data['cm_dummy'])/(1-data['propensity_score']))*data['gamedummy']) \
        / sum((1-data['cm_dummy'])/(1-data['propensity_score']))
ATE = E1 - E2
print('ATE = {:.4f}'.format(ATE))

ATE = 0.0266
